In [1]:
import os
from time import sleep
import time
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

##########################################################################
##################### variable related selenium ##########################
##########################################################################
# 수원시, 용인시, 화성시 구 리스트
gu_list = ['영통구']
# csv 파일에 헤더 만들어 주기
for index, gu_name in enumerate(gu_list):
    fileName = 'test.csv' # index.__str__() + '_' + gu_name + '.'+'csv'
    file = open(fileName, 'w', encoding='utf-8')
    file.write("카페명" + "|" + "주소" + "|" + "영업시간" + "|" + "전화번호" + "|" + "대표사진주소" + "\n")
    file.close()                                    # 처음에 csv파일에 칼럼명 만들어주기
  
    options = webdriver.ChromeOptions()
    # options.add_argument('headless')
    options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36   ")
    options.add_argument('lang=ko_KR')
    chromedriver_path = "/Users/im-yeongje/python/chromedriver"
    driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기
    driver.get('https://map.kakao.com/')  # 주소 가져오기
    search_area = driver.find_element(By.XPATH, r'//*[@id="search.keyword.query"]') # 검색 창
    search_area.send_keys(gu_name + ' 카페')  # 검색어 입력
    driver.find_element(By.XPATH, r'//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    driver.implicitly_wait(3) # 기다려 주자
    more_page = driver.find_element(By.ID, r"info.search.place.more")
    # more_page.click()
    more_page.send_keys(Keys.ENTER) # 더보기 누르고
    # 첫 번째 검색 페이지 끝
    # driver.implicitly_wait(5) # 기다려 주자
    time.sleep(1)

    # next 사용 가능?
    next_btn = driver.find_element(By.ID, "info.search.page.next")
    has_next = "disabled" not in next_btn.get_attribute("class").split(" ")
    Page = 1
    while has_next: # 다음 페이지가 있으면 loop
    # for i in range(2, 6): # 2, 3, 4, 5
        file = open(fileName, 'a', encoding='utf-8')
        time.sleep(1)
        # place_lists = driver.find_elements_by_css_selector('#info\.search\.place\.list > li:nth-child(1)')
        # 페이지 루프
        #info\.search\.page\.no1 ~ .no5
        page_links = driver.find_elements(By.CSS_SELECTOR, "#info\.search\.page a")
        pages = [link for link in page_links if "HIDDEN" not in link.get_attribute("class").split(" ")]
        # print(len(pages), "개의 페이지 있음")
        # pages를 하나씩 클릭하면서
        for i in range(1, 6):
            xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
            try:
                page = driver.find_element(By.XPATH, xPath)
                page.send_keys(Keys.ENTER)
            except ElementNotInteractableException:
                print('End of Page')
                break;
            sleep(3)
            place_lists = driver.find_elements(By.CSS_SELECTOR, '#info\.search\.place\.list > li')
            for p in place_lists: # WebElement
                # print(p.get_attribute('innerHTML'))
                # print("type of p:", type(p))
                store_html = p.get_attribute('innerHTML')
                store_info = BeautifulSoup(store_html, "html.parser")
                # BS -> 분석
                #
                place_name = store_info.select('.head_item > .tit_name > .link_name')
                # place_address = store_info.select('.info_item > .addr > p')
                # place_hour = store_info.select('.info_item > .openhour > p > a')
                # place_tel = store_info.select('.info_item > .contact > span')
                 # print("length:", len(place_name))
                if len(place_name) == 0:
                    continue # 광고
                place_name = store_info.select('.head_item > .tit_name > .link_name')[0].text
                place_address = store_info.select('.info_item > .addr > p')[0].text
                place_hour = store_info.select('.info_item > .openhour > p > a')[0].text
                place_tel = store_info.select('.info_item > .contact > span')[0].text
                 # 사진url 수집
                detail = p.find_element(By.CSS_SELECTOR, 'div.info_item > div.contact > a.moreview')
                detail.send_keys(Keys.ENTER)

                driver.switch_to.window(driver.window_handles[-1])

                place_photo = ""
                try:
                    photo = driver.find_element(By.CSS_SELECTOR, 'span.bg_present')
                    photo_url = photo.get_attribute('style')
                    m = re.search('"(.+?)"', photo_url)
                    if m:
                        place_photo = m.group(1)
                    else:
                        place_photo = ""
                except:
                    place_photo = ""
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                print(place_name, place_photo)
                file.write(place_name + "|" + place_address + "|" + place_hour + "|" + place_tel + "|" + place_photo + "\n")
            print(i, ' of', ' [ ' , Page, ' ] ')
        next_btn = driver.find_element(By.ID, "info.search.page.next")
        has_next = "disabled" not in next_btn.get_attribute("class").split(" ")
        if not has_next:
            print('Arrow is Disabled')
            driver.close()
            file.close()
            break # 다음 페이지 없으니까 종료
        else: # 다음 페이지 있으면
            Page += 1
            next_btn.send_keys(Keys.ENTER)
    print("End of Crawl")





/var/folders/0z/_2xlgtcn1w9gckgd5b2_qxlc0000gn/T/ipykernel_77524/2396495086.py:29: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기


책발전소 광교점 //t1.kakaocdn.net/thumb/T800x0.q80/?fname=http%3A%2F%2Ft1.daumcdn.net%2Fplace%2FE5A525AD48A143BEA33381A7E80E135A
커피 바이브 //t1.kakaocdn.net/thumb/T800x0.q80/?fname=http%3A%2F%2Ft1.daumcdn.net%2Flocalfiy%2Fsearchregister_1157753250
카페드단아한 광교본점 //t1.kakaocdn.net/thumb/T800x0.q80/?fname=http%3A%2F%2Ft1.daumcdn.net%2Fplace%2F8111F079B70F4DA4BE759131CD58DE7A
블레스롤 광교카페거리점 //t1.kakaocdn.net/thumb/T800x0.q80/?fname=http%3A%2F%2Ft1.daumcdn.net%2Fplace%2F17806262ECE94B1A8B1190C1DD351382
프로마쥬블랑 //t1.kakaocdn.net/thumb/T800x0.q80/?fname=http%3A%2F%2Ft1.daumcdn.net%2Fplace%2FD54A37CA3057400B92FEF92689E4D0D0
쿠로이시로 //t1.kakaocdn.net/thumb/T800x0.q80/?fname=http%3A%2F%2Ft1.daumcdn.net%2Flocalfiy%2FFF2DF2BB186D4F28ABE8968412137BBD
백금당 수원광교점 //t1.kakaocdn.net/thumb/T800x0.q80/?fname=http%3A%2F%2Ft1.kakaocdn.net%2Fmystore%2F61B4F4D0C4DB4B7380A1316B92D4BDB6
카페그루비 //t1.kakaocdn.net/thumb/T800x0.q80/?fname=http%3A%2F%2Ft1.daumcdn.net%2Fplace%2F73906C0FEE2D4F62BFF3931CD98DF0CB
카페아라 //t1.kakaocdn.net/t

MaxRetryError: HTTPConnectionPool(host='localhost', port=53937): Max retries exceeded with url: /session/99e7ec6f293aa3726fcd4465ef07e446/window (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x111ce90c0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [12]:
import os
import re
from time import sleep
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
from selenium.webdriver.common.by import By

def extract_review(place_name):
    global driver

    ret = True

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 첫 페이지 리뷰 목록 찾기
    review_lists = soup.select('.list_evaluation > li')

    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for i, review in enumerate(review_lists):
            comment = review.select('.txt_comment > span') # 리뷰
            rating = review.select('.grade_star > em') # 별점
            val = ''
            if len(comment) != 0:
                if len(rating) != 0:
                    val = comment[0].text + '/' + rating[0].text.replace('점', '')
                else:
                    val = comment[0].text + '/0'
                print(val)

    else:
        print('no review in extract')
        ret = False

    return ret



def start_search(driver, cafe_info):
    url = "https://www.google.com/maps/"
    # driverPath = "D:\workspace\pythonProject2\chromedriver.exe"
    # driver = webdriver.Chrome(driverPath)
    driver.get(url)
    # result = requests.get(url=url) # 응답

    # search = driver.find_element_by_css_selector("input#searchboxinput.tactile-searchbox-input")
    search = driver.find_element(By.CSS_SELECTOR, "#searchboxinput")
    time.sleep(1)
    search.clear()
    search.send_keys(cafe_info.get("name"))
    search.send_keys(Keys.ENTER)
    # driver.implicitly_wait(3)

    return get_store_review_data(driver, cafe_info)


def get_store_review_data(driver, cafe_info):
    while True:
        try:
            time.sleep(5)
            # 더보기 = driver.find_element_by_css_selector('#pane > div > div.widget-pane-content.scrollable-y > div > div > div:nth-child(44) > div > div > button')
            more = driver.find_element(By.CSS_SELECTOR, "button[aria-label*='리뷰 더보기']")
            more.send_keys(Keys.ENTER)
            # 더보기 = driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[41]/div/div/button')
            # time.sleep(1)
            # 더보기.click()
        except Exception as e:
            print(e)
            break

    # last_height = driver.execute_script("return document.body.scrollHeight")
    for i in range(100):
        # 스크롤 : 마지막까지
        reviews = driver.find_elements(By.CSS_SELECTOR, 'span.data-review-id')
        for review in reviews:
            print(review.text)
        print("========================================================================================================")
    

        try:
            scroll = driver.find_element(By.CSS_SELECTOR, 'div.section-layout.section-scrollbox.scrollable-y.scrollable-show')
            time.sleep(1)
            driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scroll)
            location1 = scroll.location_once_scrolled_into_view
            print(i, location1)
        except Exception as e:
            print(e)
            break

        time.sleep(1)

    # 스크롤 끝났으니 수집
    reviews = driver.find_elements(By.CSS_SELECTOR, 'span.data-review-text')
    print(reviews)
    stars = driver.find_elements(By.CSS_SELECTOR, 'span.data-review-stars')
    result =[[cafe_info.get("gu"), cafe_info.get("cafe_id"), re.search(r'[0-9]', star.get_attribute("aria-label")).group(), review.text.replace("\n", " ")] for
              star, review in zip(stars, reviews)]
    return result

    # return (stars.get_attribute("aria-label"), reviews.text) //*[@id="ChZDSUhNMG9nS0VJQ0FnSUNXNzRHUWZREAE"]/span[2]
    # print(stars.get_attribute("aria-label"), reviews.text)

def main():

    driverPath = "/Users/im-yeongje/python/chromedriver"
    driver = webdriver.Chrome(driverPath)
    # 카페 1개의 정보 구, ID, 카페명, 주소 ...
    # df = pd.read_csv(r'C:\Users\USER\Desktop\cafe_list\result_gu.csv')
    # df['검색어'] = df['카페명'] + df['주소']
    df = pd.read_csv(r'/Users/im-yeongje/python/test.csv', encoding='utf-8-sig', sep="|")
    #gu = df['구']
    #cafe_id = df['ID']
    cafe_name = df['카페명']
    cafe_addr = df['주소']
    cafe_tel = df['전화번호'].astype(str)
    cafe_len = len(cafe_name)
    search_name = []
    for i in range(cafe_len):
        name = cafe_name[i] #+ " " + cafe_addr[i] + " " + cafe_tel[i]
        search_name.append({
            "name":name,
            #"gu": gu[i],
            #"cafe_id": cafe_id[i]
        })

    for i in range(cafe_len):

        result = start_search(driver, search_name[i])

        # 구, ID, stars, 리뷰
        # result = get_store_review_data(driver)
        data = pd.DataFrame(result)
        # data.columns = ['stars', 'reviews']
        # data.columns = ['stars', 'reviews']
        # data = pd.concat([search_name[i].get("gu") + search_name[i].get("cafe_id"), data], axis=1)
        # 누적
        if not os.path.exists("cafe_reviews_GA_GC.csv"):
            data.to_csv('cafe_reviews.csv', index=False, sep='|', mode="w")
        else:
            data.to_csv('cafe_reviews.csv', index=False, sep='|', mode="a", header=False)



"""
1. 카페 정보 불러오기
2. 한라인씩 읽으면 -> 해당 카페의 식별정보(구, ID)
3. 수집 -> [구, ID, stars, review]
"""


if __name__ == "__main__":
    main()


/var/folders/0z/_2xlgtcn1w9gckgd5b2_qxlc0000gn/T/ipykernel_79668/3563902922.py:115: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driverPath)


Message: no such element: Unable to locate element: {"method":"css selector","selector":"button[aria-label*='리뷰 더보기']"}
  (Session info: chrome=106.0.5249.119)
Stacktrace:
0   chromedriver                        0x0000000100c02f44 chromedriver + 3813188
1   chromedriver                        0x0000000100b94ae0 chromedriver + 3361504
2   chromedriver                        0x0000000100898c5c chromedriver + 232540
3   chromedriver                        0x00000001008ca380 chromedriver + 435072
4   chromedriver                        0x00000001008f7f44 chromedriver + 622404
5   chromedriver                        0x00000001008bf358 chromedriver + 389976
6   chromedriver                        0x00000001008c0284 chromedriver + 393860
7   chromedriver                        0x0000000100bd6e18 chromedriver + 3632664
8   chromedriver                        0x0000000100bda97c chromedriver + 3647868
9   chromedriver                        0x0000000100be0b58 chromedriver + 3672920
10  chromedri

KeyboardInterrupt: 

In [7]:
import os
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup

##########################################################################
##################### variable related selenium ##########################
##########################################################################
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('lang=ko_KR')
chromedriver_path = "/Users/im-yeongje/python/chromedriver"
driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기

#개인적 코드(수언 카페 추가)
gu_list = ['영통구']

def main():
    global driver, load_wb, review_num

    driver.implicitly_wait(4)  # 렌더링 될때까지 기다린다 4초
    driver.get("https://www.google.com/maps/")  # 주소 가져오기

    # 검색할 목록
    # place_infos = ['강남 맛집']

    for i, place in enumerate(gu_list):
        # delay
        if i % 4 == 0 and i != 0:
            sleep(5)
        print("#####", i)
        search(place + '카페')

    driver.quit()
    print("finish")


def search(place):
    global driver

    search_area = driver.find_element(By.XPATH, r'//*[@id="search.keyword.query"]')  # 검색 창
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element(By.XPATH, r'//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    sleep(1)

    # 검색된 정보가 있는 경우에만 탐색
    # 1번 페이지 place list 읽기
    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')
    place_lists = soup.select('.placelist > .PlaceItem') # 검색된 장소 목록

    # 검색된 첫 페이지 장소 목록 크롤링하기
    crawling(place, place_lists)
    search_area.clear()

    # 우선 더보기 클릭해서 2페이지
    try:
        driver.find_element(By.XPATH, r'//*[@id="info.search.place.more"]').send_keys(Keys.ENTER)
        sleep(1)

        # 2~ 5페이지 읽기
        for i in range(2, 6):
            # 페이지 넘기기
            xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
            driver.find_element(By.XPATH, xPath).send_keys(Keys.ENTER)
            sleep(1)

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            place_lists = soup.select('.placelist > .PlaceItem') # 장소 목록 list

            crawling(place, place_lists)

    except ElementNotInteractableException:
        print('not found')
    finally:
        search_area.clear()


def crawling(place, place_lists):
    """
    페이지 목록을 받아서 크롤링 하는 함수
    :param place: 리뷰 정보 찾을 장소이름
    """

    while_flag = False
    for i, place in enumerate(place_lists):
        # 광고에 따라서 index 조정해야함
        #if i >= 3:
         #   i += 1

        place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
        place_address = place.select('.info_item > .addr > p')[0].text  # place address

        detail_page_xpath = '//*[@id="info.search.place.list"]/li[' + str(i + 1) + ']/div[5]/div[4]/a[1]'
        driver.find_element(By.XPATH, detail_page_xpath).send_keys(Keys.ENTER)
        driver.switch_to.window(driver.window_handles[-1])  # 상세정보 탭으로 변환
        sleep(1)

        print('####', place_name)

        # 첫 페이지
        extract_review(place_name)

        # 2-5 페이지
        idx = 3
        try:
            page_num = len(driver.find_elements(By.CLASS_NAME, 'link_page')) # 페이지 수 찾기
            for i in range(page_num-1):
                # css selector를 이용해 페이지 버튼 누르기
                driver.find_element(By.CSS_SELECTOR, '#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                sleep(1)
                extract_review(place_name)
                idx += 1
            driver.find_element(By.LINK_TEXT, '다음').send_keys(Keys.ENTER) # 5페이지가 넘는 경우 다음 버튼 누르기
            sleep(1)
            extract_review(place_name) # 리뷰 추출
        except (NoSuchElementException, ElementNotInteractableException):
            print("no review in crawling")

        # 그 이후 페이지
        while True:
            idx = 4
            try:
                page_num = len(driver.find_elements(By.CLASS_NAME, 'link_page'))
                for i in range(page_num-1):
                    driver.find_element(By.CSS_SELECTOR, '#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                    sleep(1)
                    extract_review(place_name)
                    idx += 1
                driver.find_element(By.LINK_TEXT, '다음').send_keys(Keys.ENTER) # 10페이지 이상으로 넘어가기 위한 다음 버튼 클릭
                sleep(1)
                extract_review(place_name) # 리뷰 추출
            except (NoSuchElementException, ElementNotInteractableException):
                print("no review in crawling")
                break

        driver.close()
        driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환


def extract_review(place_name):
    global driver

    ret = True

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 첫 페이지 리뷰 목록 찾기
    review_lists = soup.select('.list_evaluation > li')

    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for i, review in enumerate(review_lists):
            comment = review.select('.txt_comment > span') # 리뷰
            rating = review.select('.grade_star > em') # 별점
            val = ''
            if len(comment) != 0:
                if len(rating) != 0:
                    val = comment[0].text + '/' + rating[0].text.replace('점', '')
                else:
                    val = comment[0].text + '/0'
                print(val)

    else:
        print('no review in extract')
        ret = False

    return ret


if __name__ == "__main__":
    main()


/var/folders/0z/_2xlgtcn1w9gckgd5b2_qxlc0000gn/T/ipykernel_79668/2562678841.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(os.path.join(os.getcwd(), chromedriver_path), options=options)  # chromedriver 열기


##### 0


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="search.keyword.query"]"}
  (Session info: headless chrome=106.0.5249.119)
Stacktrace:
0   chromedriver                        0x00000001023f2f44 chromedriver + 3813188
1   chromedriver                        0x0000000102384ae0 chromedriver + 3361504
2   chromedriver                        0x0000000102088c5c chromedriver + 232540
3   chromedriver                        0x00000001020ba380 chromedriver + 435072
4   chromedriver                        0x00000001020e7f44 chromedriver + 622404
5   chromedriver                        0x00000001020af358 chromedriver + 389976
6   chromedriver                        0x00000001020b0284 chromedriver + 393860
7   chromedriver                        0x00000001023c6e18 chromedriver + 3632664
8   chromedriver                        0x00000001023ca97c chromedriver + 3647868
9   chromedriver                        0x00000001023d0b58 chromed